In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import pandasql as pds
import os
import re
import datetime as dt
import numpy as np

In [9]:
odf = pd.read_csv('presidents.csv')
odf

,#,President,Born,Age atstart of presidency,Age atend of presidency,Post-presidencytimespan,Died,Age
0,1,George Washington,"Feb 22, 1732[a]","57 years, 67 daysApr 30, 1789","65 years, 10 daysMar 4, 1797","2 years, 285 days","Dec 14, 1799","67 years, 295 days"
1,2,John Adams,"Oct 30, 1735[a]","61 years, 125 daysMar 4, 1797","65 years, 125 daysMar 4, 1801","25 years, 122 days","Jul 4, 1826","90 years, 247 days"
2,3,Thomas Jefferson,"Apr 13, 1743[a]","57 years, 325 daysMar 4, 1801","65 years, 325 daysMar 4, 1809","17 years, 122 days","Jul 4, 1826","83 years, 82 days"
3,4,James Madison,"Mar 16, 1751[a]","57 years, 353 daysMar 4, 1809","65 years, 353 daysMar 4, 1817","19 years, 116 days","Jun 28, 1836","85 years, 104 days"
4,5,James Monroe,"Apr 28, 1758","58 years, 310 daysMar 4, 1817","66 years, 310 daysMar 4, 1825","6 years, 122 days","Jul 4, 1831","73 years, 67 days"
5,6,John Quincy Adams,"Jul 11, 1767","57 years, 236 daysMar 4, 1825","61 years, 236 daysMar 4, 1829","18 years, 356 days","Feb 23, 1848","80 years, 227 days"
6,7,Andrew Jackson,"Mar 15, 1767","61 years, 354 daysMar 4, 1829","69 years, 354 daysMar 4, 1837","8 years, 96 days","Jun 8, 1845","78 years, 85 days"
7,8,Martin Van Buren,"Dec 5, 1782","54 years, 89 daysMar 4, 1837","58 years, 89 daysMar 4, 1841","21 years, 142 days","Jul 24, 1862","79 years, 231 days"
8,9,William H. Harrison,"Feb 9, 1773","68 years, 23 daysMar 4, 1841","68 years, 54 days Apr 4, 1841[b]",NaN,"Apr 4, 1841","68 years, 54 days"
9,10,John Tyler,"Mar 29, 1790","51 years, 6 daysApr 4, 1841","54 years, 340 daysMar 4, 1845","16 years, 320 days","Jan 18, 1862","71 years, 295 days"


In [7]:
df = odf.rename(mapper = str.strip, axis = 'columns')

del df['#']

pat_b = "(\w{3} \d{1,2}, \d{4})"
df['Born'].str.extract(pat_b)
df['Born'] = df['Born'].str.extract(pat_b)

pat_aa = "(.*)(?=[A-Z].*)"
df['Age atstart of presidency'].str.extract(pat_aa)
df['AgeAtStart'] = df['Age atstart of presidency'].str.extract(pat_aa)
pat_ab = "(?=.*)([A-Z].*\d$)"
df['StartOfPresidency'] = df['Age atstart of presidency'].str.extract(pat_ab)

end = 'Age atend of presidency'
df['AgeAtEnd'] = df[end].str.extract(pat_aa)
df['EndOfPresidency'] = df[end].str.extract(pat_ab)
del df[end]
del df['Age atstart of presidency']

df.rename(columns = {'Post-presidencytimespan':'PostPresidency'}, inplace = True)


df['PostPresidency'].fillna(0, inplace = True)
pat = '(.*)(?=\[\w\])'
df['EndOfPresidency'].fillna(df['Died'], inplace = True)

df['Born'] = df['Born'].apply(lambda x: dt.datetime.strptime(x, ("%b %d, %Y")))

df['StartOfPresidency'] = df['StartOfPresidency'].apply(lambda x: dt.datetime.strptime(x, ("%b %d, %Y")))
df['EndOfPresidency'] = df['EndOfPresidency'].apply(lambda x: dt.datetime.strptime(x, ("%b %d, %Y")))

diedlist = df['Died'].tolist()
for i in range(len(diedlist)):
    if diedlist[i] == '(living)':
        continue  
    diedlist[i] = dt.datetime.strptime(diedlist[i], ("%b %d, %Y"))
    
df['Died'] = pd.Series(diedlist)

df.query("Died == '(living)'")

aadf = pd.read_csv('presidents.csv')  # not lambda, actual function and then the condition inside the function and then instead of replace you map it on the column(series).

df

,President,Born,PostPresidency,Died,Age,AgeAtStart,StartOfPresidency,AgeAtEnd,EndOfPresidency
0,George Washington,1732-02-22,"2 years, 285 days",1799-12-14 00:00:00,"67 years, 295 days","57 years, 67 days",1789-04-30,"65 years, 10 days",1797-03-04
1,John Adams,1735-10-30,"25 years, 122 days",1826-07-04 00:00:00,"90 years, 247 days","61 years, 125 days",1797-03-04,"65 years, 125 days",1801-03-04
2,Thomas Jefferson,1743-04-13,"17 years, 122 days",1826-07-04 00:00:00,"83 years, 82 days","57 years, 325 days",1801-03-04,"65 years, 325 days",1809-03-04
3,James Madison,1751-03-16,"19 years, 116 days",1836-06-28 00:00:00,"85 years, 104 days","57 years, 353 days",1809-03-04,"65 years, 353 days",1817-03-04
4,James Monroe,1758-04-28,"6 years, 122 days",1831-07-04 00:00:00,"73 years, 67 days","58 years, 310 days",1817-03-04,"66 years, 310 days",1825-03-04
5,John Quincy Adams,1767-07-11,"18 years, 356 days",1848-02-23 00:00:00,"80 years, 227 days","57 years, 236 days",1825-03-04,"61 years, 236 days",1829-03-04
6,Andrew Jackson,1767-03-15,"8 years, 96 days",1845-06-08 00:00:00,"78 years, 85 days","61 years, 354 days",1829-03-04,"69 years, 354 days",1837-03-04
7,Martin Van Buren,1782-12-05,"21 years, 142 days",1862-07-24 00:00:00,"79 years, 231 days","54 years, 89 days",1837-03-04,"58 years, 89 days",1841-03-04
8,William H. Harrison,1773-02-09,0,1841-04-04 00:00:00,"68 years, 54 days","68 years, 23 days",1841-03-04,"68 years, 54 days",1841-04-04
9,John Tyler,1790-03-29,"16 years, 320 days",1862-01-18 00:00:00,"71 years, 295 days","51 years, 6 days",1841-04-04,"54 years, 340 days",1845-03-04
